## Movie Recommendation System

Öneri sistemleri , kullanıcılara çeşitli kriterlere göre önerilerde bulunan bilgisayar programlarıdır.

Bu sistemler tüketicilerin satın alacakları ve ilgilenecekleri en olası ürünü tahmin ediyor. Netflix, Amazon ve diğer şirketler, kullanıcılarının kendileri için doğru ürünü veya filmi bulmalarına yardımcı olmak için öneri sistemlerini kullanıyor. Bu sistemler, izleme veya satın alma geçmişinizdeki kalıpları öğrenerek ve anlayarak önerilerde bulunur, ardından yeni önerilerde bulunmak için bu kalıpları ve bulguları uygular.

3 tip öneri sistemi vardır.

1.   Demografik Filtreleme:  Öneriler her kullanıcı için aynıdır. Kişiselleştirilmezler, genelleştirilirler. Bu tür sistemler "En Trend" gibi bölümlerin gerisindedir.
2.   İçerik Tabanlı Filtreleme: Bunlar, öğe meta verilerine (film, ürün, şarkı vb.) dayalı öneriler önerir. Burada ana fikir, bir kullanıcı bir öğeyi beğenirse, kullanıcı buna benzer öğeleri de sevecektir.
3.   İşbirliğine Dayalı Filtreleme: Bu sistemler, benzer ilgi alanlarına sahip kullanıcıları gruplayarak önerilerde bulunur. Bu sistem için öğenin meta verileri gerekli değildir.

Bu makine öğrenimi projesinde, kullanıcıya tercihlerine göre film önermek için sıfırdan bir öneri sistemi oluşturuyoruz. Bu projede İçerik Tabanlı Filtreleme kullanıyoruz.

Film öneri motorunu oluşturma yaklaşımı aşağıdaki adımlardan oluşur.

*   Veriler üzerinde Keşifsel Veri Analizi (EDA) gerçekleştirilmesi
*   Öneri sistemini oluşturulması
*   Önerilerin alınması

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

## **Adım 1: Veriler üzerinde Keşifsel Veri Analizi (EDA) gerçekleştirilmesi**

 Ancak bu proje için filmin meta verilerini (oyuncu kadrosu, ekip, bütçe vb.) içeren bir veri seti kullanacağız. The Movie Database kullanılarak oluşturulmuştur. 
 
Dataset: https://www.kaggle.com/tmdb/tmdb-movie-metadata

Drive linki: https://drive.google.com/drive/folders/1zg3KQWlAZpO6CWxnB7-AduwOxX5B_INZ?usp=sharing

Veri kümesi iki farklı CSV dosyası, kredi ve film içerir. Credits dosyası, filmle ilgili tüm meta veri bilgilerini içerir ve film dosyası, filmin adı ve kimliği, bütçe, gösterime giren filmdeki diller gibi bilgileri içerir.

Pandas kütüphanesini kullanarak film veri setini yükleyelim.

In [3]:
credits_df = pd.read_csv("https://raw.githubusercontent.com/kaveai/veribilimiyazokulu/main/Ders%20%C4%B0%C3%A7erikleri/tmdb_5000_credits.csv")
movies_df = pd.read_csv("https://raw.githubusercontent.com/kaveai/veribilimiyazokulu/main/Ders%20%C4%B0%C3%A7erikleri/tmdb_5000_movies.csv")

Veri çerçevelerimize bir göz atalım.

In [4]:
movies_df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
credits_df.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


Yalnızca kredi verisetindeki id, title, cast ve crew sütunlarına ihtiyacımız var. Verisetlerini 'id' sütununu kullanarak bir araya getirelim.

In [6]:
credits_df.columns = ['id','tittle','cast','crew']
movies_df = movies_df.merge(credits_df, on="id")

In [7]:
movies_df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


## **2. Adım: Film Öneri Sistemini Oluşturun**

Öneri sistemi tarafından yapılan tahminlerin doğruluğu, filmin “konu/açıklaması” kullanılarak kişiselleştirilebilir. Ancak, filmin meta verileri kullanılarak önerilerin kalitesi daha da geliştirilebilir. Film tavsiye motorumuza gelen sorgunun “The Dark Knight Rises” olduğunu varsayalım. O zaman tahminler, filmin yönetmeninin yönettiği filmleri de içermelidir. Ayrıca verilen sorgu, filmin oyuncu kadrosuna sahip filmleri de içermelidir.

Bunun için tavsiyeyi kişiselleştirmek için şu özellikleri kullanıyoruz: cast, crev, keyword, genres.

Film verileri diziler içeren listeler şeklinde bulunur, verileri güvenli ve kullanılabilir bir yapıya dönüştürmemiz gerekir. literal_eval() fonksiyonunu özelliklere uygulayalım.

In [8]:
features = ["cast", "crew", "keywords", "genres"]

for feature in features:
    movies_df[feature] = movies_df[feature].apply(literal_eval)

movies_df[features].head(10)

,cast,crew,keywords,genres
0,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
1,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
3,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
4,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
5,"[{'cast_id': 30, 'character': 'Peter Parker / ...","[{'credit_id': '52fe4252c3a36847f80151a5', 'de...","[{'id': 851, 'name': 'dual identity'}, {'id': ...","[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na..."
6,"[{'cast_id': 34, 'character': 'Flynn Rider (vo...","[{'credit_id': '52fe46db9251416c91062101', 'de...","[{'id': 1562, 'name': 'hostage'}, {'id': 2343,...","[{'id': 16, 'name': 'Animation'}, {'id': 10751..."
7,"[{'cast_id': 76, 'character': 'Tony Stark / Ir...","[{'credit_id': '55d5f7d4c3a3683e7e0016eb', 'de...","[{'id': 8828, 'name': 'marvel comic'}, {'id': ...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
8,"[{'cast_id': 3, 'character': 'Harry Potter', '...","[{'credit_id': '52fe4273c3a36847f801fab1', 'de...","[{'id': 616, 'name': 'witch'}, {'id': 2343, 'n...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
9,"[{'cast_id': 18, 'character': 'Bruce Wayne / B...","[{'credit_id': '553bf23692514135c8002886', 'de...","[{'id': 849, 'name': 'dc comics'}, {'id': 7002...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."


Yukarıdaki özelliklerden yönetmen gibi bilgileri çıkarmak için bazı fonksiyonlar yazalım.

In [9]:
movies_df["crew"][0]

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

get_director() işlevi, filmin yönetmeninin adını çıkarır.

In [10]:
def get_director(x):
    for i in x:
        if i["job"] == "Director":
            return i["name"]
    return np.nan

get_list(), listenin uzunluğu 3'den daha fazlaysa, ilk 3 öğeyi veya tüm listeyi döndürür.

In [11]:
movies_df["cast"][0]

[{'cast_id': 242,
  'character': 'Jake Sully',
  'credit_id': '5602a8a7c3a3685532001c9a',
  'gender': 2,
  'id': 65731,
  'name': 'Sam Worthington',
  'order': 0},
 {'cast_id': 3,
  'character': 'Neytiri',
  'credit_id': '52fe48009251416c750ac9cb',
  'gender': 1,
  'id': 8691,
  'name': 'Zoe Saldana',
  'order': 1},
 {'cast_id': 25,
  'character': 'Dr. Grace Augustine',
  'credit_id': '52fe48009251416c750aca39',
  'gender': 1,
  'id': 10205,
  'name': 'Sigourney Weaver',
  'order': 2},
 {'cast_id': 4,
  'character': 'Col. Quaritch',
  'credit_id': '52fe48009251416c750ac9cf',
  'gender': 2,
  'id': 32747,
  'name': 'Stephen Lang',
  'order': 3},
 {'cast_id': 5,
  'character': 'Trudy Chacon',
  'credit_id': '52fe48009251416c750ac9d3',
  'gender': 1,
  'id': 17647,
  'name': 'Michelle Rodriguez',
  'order': 4},
 {'cast_id': 8,
  'character': 'Selfridge',
  'credit_id': '52fe48009251416c750ac9e1',
  'gender': 2,
  'id': 1771,
  'name': 'Giovanni Ribisi',
  'order': 5},
 {'cast_id': 7,
  'c

In [12]:
def get_list(x):
    if isinstance(x, list): #x'in tipi liste mi?
        names = [i["name"] for i in x]

        if len(names) > 3:
            names = names[:3]

        return names

    return []

Veri kümemize hem get_director() hem de get_list() işlevlerini uygulayalım.

In [13]:
movies_df["director"] = movies_df["crew"].apply(get_director)

features = ["cast", "keywords", "genres"]
for feature in features:
    movies_df[feature] = movies_df[feature].apply(get_list)

Yukarıda “crew” bilgisini get_director() fonksiyonuna ilettik, ismi çıkarttık ve yeni bir “director” kolonu oluşturduk.

genres, keywords ve cast için get_list() işlevini uygulayarak en önemli bilgileri çıkardık.

Yukarıdaki dönüşümlerden sonra verilerin nasıl göründüğünü görelim.

In [14]:
movies_df[['title', 'cast', 'director', 'keywords', 'genres']].head(2)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"


Bir sonraki adım, yukarıdaki özellik örneklerini küçük harfe dönüştürmek ve aralarındaki tüm boşlukları kaldırmak olacaktır.

In [15]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ""

In [16]:
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    movies_df[feature] = movies_df[feature].apply(clean_data)

Şimdi, Vektorizer'a girmek için çıkarılan tüm meta veri bilgilerini içeren bir soup oluşturalım.

In [17]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


movies_df["soup"] = movies_df.apply(create_soup, axis=1)
print(movies_df["soup"].head(2))

0    cultureclash future spacewar samworthington zo...
1    ocean drugabuse exoticisland johnnydepp orland...
Name: soup, dtype: object


Film öneri motorumuz, meta veri bilgilerine dayalı olarak kullanıcıya filmler önererek çalışır. Filmler arasındaki benzerlik hesaplanır ve daha sonra önerilerde bulunmak için kullanılır. Bunun için metin verilerimiz önceden işlenmeli ve CountVectorizer kullanılarak bir vektörleştiriciye dönüştürülmelidir. Adından da anlaşılacağı gibi, CountVectorizer her kelimenin sıklığını sayar ve frekansları içeren bir 2B vektör çıktısı verir.

a, an, the gibi kelimeleri (bunlara “durdurma kelimeleri/stopwords” denir) dikkate almayız çünkü bu kelimeler genellikle metinde daha yüksek miktarlarda bulunur ve herhangi bir anlam ifade etmez.

Kosinüs benzerliği, Pearson korelasyon katsayısı vb. gibi birkaç benzerlik puanı işlevi vardır. Burada, CountVectorizer tarafından vektör çıktısının nokta çarpımı olduğu için kosinüs benzerlik puanını kullanırız.

Ayrıca veri çerçevemizin indekslerini de sıfırlıyoruz.

In [18]:
count_vectorizer = CountVectorizer(stop_words="english")
count_matrix = count_vectorizer.fit_transform(movies_df["soup"])

print(count_matrix.shape)

cosine_sim_2 = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim_2.shape)

movies_df = movies_df.reset_index()
indices = pd.Series(movies_df.index, index=movies_df['title'])

(4803, 11520)
(4803, 4803)


Film başlıklarının dizinlerle ters eşlemesini oluşturun. Bu sayede filmin adını dizine göre kolayca bulabiliriz.

In [19]:
indices = pd.Series(movies_df.index, index=movies_df["title"]).drop_duplicates()
print(indices.head(2))

title
Avatar                                      0
Pirates of the Caribbean: At World's End    1
dtype: int64


### **3. Adım: Filmler için öneriler alın**

get_recommendations() işlevi, filmin başlığını ve benzerlik işlevini girdi olarak alır. Önerilerde bulunmak için aşağıdaki adımları takip eder.

Başlığı kullanarak filmin dizinini alın.
Tüm filmlerle ilgili filmlerin benzerlik puanlarının listesini alıyoruz.
İlk eleman indeks ve ikinci eleman kosinüs benzerlik puanı olacak şekilde onları numaralandırıyoruz(tuples oluşturuyoruz).
Benzerlik puanına göre grup listesini azalan düzende sıralayıp,
yukarıda sıralanan listeden en iyi 10 filmin endekslerinin listesini alıyoruz. Başlığın kendisi olduğu için ilk öğeyi hariç tuttuk.
Bu dizinleri ilgili başlıklarla eşleştirip film listesini döndürüyoruz.
Film başlığını ve kosinüs benzerlik puanını girdi olarak alan ve buna benzer ilk 10 filmi çıkaran bir işlev oluşturuyoruz.

In [20]:
def get_recommendations(title, cosine_sim=cosine_sim_2):
    idx = indices[title]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores= sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores= similarity_scores[1:11]
    # (a, b) where a is id of movie, b is similarity_scores
    movies_indices = [ind[0] for ind in similarity_scores]
    movies = movies_df["title"].iloc[movies_indices]
    return movies

In [21]:
print("################ Content Based System #############")
print("Recommendations for The Dark Knight Rises")
print(get_recommendations("The Dark Knight Rises", cosine_sim_2))
print()
print("Recommendations for Avengers")
print(get_recommendations("The Avengers", cosine_sim_2))

################ Content Based System #############
Recommendations for The Dark Knight Rises
65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object

Recommendations for Avengers
7                  Avengers: Age of Ultron
26              Captain America: Civil War
79                              Iron Man 2
169     Captain America: The First Avenger
174                    The Incredible Hulk
85     Captain America: The Winter Soldier
31                              Iron Man 3
33                   X-Men: The Last Stand
68                                Iron Man
94                 Guardians of the Galaxy
Name: title, dtype: object


Kaynak: https://techvidvan.com/tutorials/movie-recommendation-system-python-machine-learning/